In [ ]:
#STEP1
!pip install yt_dlp
!pip install fpdf


In [ ]:
import yt_dlp
import os

# 提示用户输入视频链接
video_url = input("请输入 YouTube 视频链接并且按回车：")

# 创建一个文件夹保存字幕
os.makedirs("subtitles", exist_ok=True)

# 设置 yt_dlp 参数（只下载英文字幕，不下载视频）
ydl_opts = {
    'skip_download': True,
    'writesubtitles': True,
    'writeautomaticsub': True,   # ← 添加这一行
    'subtitleslangs': ['en'],
    'subtitlesformat': 'srt',
    'outtmpl': 'subtitles/%(title)s.%(ext)s'
}


with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    info = ydl.extract_info(video_url, download=True)
    video_title = info.get('title')
    print(f"《{video_title}》的字幕已下载。")


请输入 YouTube 视频链接并且按回车：https://www.youtube.com/watch?v=nCTD_Cf4zV0
[youtube] Extracting URL: https://www.youtube.com/watch?v=nCTD_Cf4zV0
[youtube] nCTD_Cf4zV0: Downloading webpage
[youtube] nCTD_Cf4zV0: Downloading tv client config
[youtube] nCTD_Cf4zV0: Downloading tv player API JSON
[youtube] nCTD_Cf4zV0: Downloading ios player API JSON


In [ ]:
import os

# 找到最新下载的字幕
srt_file = max(
    [os.path.join("subtitles", f) for f in os.listdir("subtitles") if f.endswith(".srt")],
    key=os.path.getmtime
)

print(f"字幕文件路径：{srt_file}")

# 打印前几行内容
with open(srt_file, 'r', encoding='utf-8') as f:
    lines = f.readlines()
    print(''.join(lines[:20]))  # 预览前20行


In [ ]:
import os

# 找到最新的 .srt 文件
srt_file = max(
    [os.path.join("subtitles", f) for f in os.listdir("subtitles") if f.endswith(".srt")],
    key=os.path.getmtime
)

# 读取并清洗字幕内容
subtitle_lines = []
with open(srt_file, 'r', encoding='utf-8') as f:
    lines = f.readlines()
    for line in lines:
        line = line.strip()
        if line.isdigit() or '-->' in line or line == '':
            continue
        subtitle_lines.append(line)

# 每 4 行合成一段
paragraphs = []
for i in range(0, len(subtitle_lines), 4):
    paragraph = ' '.join(subtitle_lines[i:i+4])
    paragraphs.append(paragraph)

# 最终文本
final_text = '\n\n'.join(paragraphs)
print(final_text[:1000000])  # 预览前1000000字


In [ ]:
import os
from fpdf import FPDF
from google.colab import files

# 1. 找到 subtitles 文件夹中最新的 .srt 文件
srt_file = max(
    [os.path.join("subtitles", f) for f in os.listdir("subtitles") if f.endswith(".srt")],
    key=os.path.getmtime
)

# 2. 清洗字幕文本
lines = []
with open(srt_file, 'r', encoding='utf-8') as f:
    for line in f:
        line = line.strip()
        if line == '' or line.isdigit() or '-->' in line:
            continue
        lines.append(line)

# 3. 每 4 行合并为一段
paragraphs = [' '.join(lines[i:i+4]) for i in range(0, len(lines), 4)]

# 4. 生成 PDF
pdf = FPDF()
pdf.add_page()
pdf.set_font("Arial", size=12)
for p in paragraphs:
    pdf.multi_cell(0, 10, p)
    pdf.ln()

# 5. 保存为 youtube.pdf
pdf.output("youtube.pdf")

# 6. 触发下载
files.download("youtube.pdf")

